In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()

In [3]:
count = 50000

while(True):
    cur = pd.read_csv("../../data/global acc/result" + str(count) + ".csv", skipinitialspace=True, header=None)
    df = df.append(cur, ignore_index=True)
    if count == 500000:
        break
    count += 50000
cur = pd.read_csv("../../data/global acc/result_final.csv", skipinitialspace=True, header=None)
df = df.append(cur, ignore_index=True)

In [4]:
len(df)

549503

In [5]:
df.columns = ['global_acc1','global_acc2','global_acc3']

In [6]:
t = pd.DataFrame.from_csv("../../data/linear_acceleration_grav_mag.csv")

In [7]:
t.columns

Index(['tester_id', 'TagName', 'Time', 'acc_val1', 'acc_val2', 'acc_val3',
       'acc_val4', 'gravity1', 'gravity2', 'gravity3', 'mag_val1', 'mag_val2',
       'mag_val3'],
      dtype='object')

In [8]:
b = t[['tester_id', 'TagName', 'Time']].copy()

In [9]:
for col in ['global_acc1','global_acc2','global_acc3']:
    b[col] = 0

b[['global_acc1','global_acc2','global_acc3']] = df[['global_acc1','global_acc2','global_acc3']].values

In [10]:
b.to_csv("../../data/global_acc_df.csv")

In [ ]:
 -0.058560904, -0.1300819, -0.520824
     -0.058560904, -0.1300819, -0.520824
        
        -0.058560826, -0.13008204, -0.52082384

In [11]:
t.head()

,tester_id,TagName,Time,acc_val1,acc_val2,acc_val3,acc_val4,gravity1,gravity2,gravity3,mag_val1,mag_val2,mag_val3
0,0.0,Tag0,2018-01-08 15:49:46.379666666+08:00,0.478294,-0.038180,-0.247761,0,-7.252295,0.090840,6.600481,8.723001,-6.4965,-51.6365
1,0.0,Tag0,2018-01-08 15:49:46.384666666+08:00,0.401698,0.028842,-0.286059,0,-7.252295,0.090840,6.600481,8.723001,-6.4965,-51.6365
2,0.0,Tag0,2018-01-08 15:49:46.389666666+08:00,0.411272,0.000118,-0.324357,0,-7.252295,0.090840,6.600481,8.143500,-6.4965,-49.1355
3,0.0,Tag0,2018-01-08 15:49:46.395666666+08:00,0.374330,-0.074288,-0.332594,0,-7.253652,0.079075,6.599143,8.143500,-6.4965,-49.1355
4,0.0,Tag0,2018-01-08 15:49:46.400666666+08:00,0.374330,-0.074288,-0.332594,0,-7.253652,0.079075,6.599143,7.564000,-7.1065,-50.3860


In [12]:
b.head()

,tester_id,TagName,Time,global_acc1,global_acc2,global_acc3
0,0.0,Tag0,2018-01-08 15:49:46.379666666+08:00,-0.058561,-0.130082,-0.520824
1,0.0,Tag0,2018-01-08 15:49:46.384666666+08:00,0.021859,-0.064001,-0.489335
2,0.0,Tag0,2018-01-08 15:49:46.389666666+08:00,-0.002340,-0.037200,-0.522459
3,0.0,Tag0,2018-01-08 15:49:46.395666666+08:00,-0.070036,0.008021,-0.501289
4,0.0,Tag0,2018-01-08 15:49:46.400666666+08:00,-0.069951,0.008737,-0.501289
